## Generic programming 

One of Julia's strongest features is its **generic programming** ability, i.e. the possibility of writing an algorithm at a high level that can run efficiently in many different contexts, abstracting away many of the implementation and even hardware details.

In this lecture and the following one, we will explore the **power method** as a test case.

### The power method

The power method is a simple algorithm (not recommended for serious use!) to calculate the largest eigenvalue of a matrix.

Recall that $(\mathbf{v}, \lambda)$ is an eigenvector / eigenvalue pair for $\mathsf{A}$ if

$$\mathsf{A} \cdot \mathbf{v} = \lambda \mathbf{v},$$

where $\lambda \in \mathbb{C}$ is a scalar, which, in general, may be a complex number.

The algorithm is as follows:

> Take an arbitrary non-zero initial vector $\mathbf{v}_0$ and apply $\mathsf{A}$ to it many times, to obtain the sequence

> $$\mathbf{v}_i := \mathsf{A}^i \cdot \mathbf{v}_0.$$

This sequence of vectors converges to the eigenvector of $\mathsf{A}$ with the largest eigenvalue.

If the $\mathbf{v}_i$ are normalised (which, in practice, they usually must be so that they do not explode during the algorithm), then the eigenvalue is given by the limit of $\mathbf{v}_n \cdot \mathsf{A} \cdot \mathbf{v}_n$. In practice, with finite precision (e.g. standard `Float64`), this limit is attained after a finite number of steps.

### Implementation in Julia

Take, for example, the matrix

In [3]:
M = [2. 1; 1 1]

2×2 Array{Float64,2}:
 2.0  1.0
 1.0  1.0

In [5]:
using LinearAlgebra

and an initial non-zero, normalized vector:

In [6]:
v = rand(2)
normalize!(v)    # only in Julia v0.5
#v /= norm(v)     # alternative: v = v / norm(v)

The basic step of the algorithm that must be repeated (use `Ctrl-Enter` in the notebook to repeatedly execute the same cell) is:

In [7]:
v = M * v
normalize!(v) 

2-element Array{Float64,1}:
 0.884540577192654  
 0.46646325396507554

To check that this is indeed an eigenvector, we do an element-wise division:

In [8]:
(M*v) ./ v

2-element Array{Float64,1}:
 2.5273508824722684
 2.896270648703403 

### Power method function

Both for ease of use and efficiency, we must now wrap this in a function:

In [14]:
function power_method(M, v)
    for i in 1:100
        v = M*v        # repeatedly creates a new vector and destroys the old v
        v/=norm(v)
    end
    
    return v, norm(M*v) / norm(v)  # or  (M*v) ./ v
end

power_method (generic function with 1 method)

We could write a version that *modifies* its argument `v`, if we so desired (although in general this is usually a bad idea) with:

In [10]:
function power_method!(M, v)
    for i in 1:100
        v[:] = normalize(M*v)   # assign in-place to the elements of v
    end
    
    return v, norm(M*v) / norm(v)
end

power_method! (generic function with 1 method)

We can now apply our function:

In [11]:
power_method(M, rand(2))

([0.850651, 0.525731], 2.618033988749895)

**Exercise**: Modify the function so that it returns once the eigenvalue has converged to the eigenvalue, rather than doing a fixed number of iterations. [Check convergence of the eigenvalue, not the eigenvector.]

## What is generic about this?

This code is **generic**: we have not specified what kind of object `M` and `v` are, i.e. what **type** of object they are. We can now try using different types of arguments:

In [15]:
power_method(3, 1)

(1.0, 3.0)

Here, we have effectively treated the number $3$ as an operator that takes $x$ and returns $3x$.  The power method is thus a complicated way to solve the equation $3x = \lambda x$ and show that the operator $3$ has eigenvalue $3$ and "eigenvector" $1$.

We could try to call the function e.g. with strings, but several things will go wrong, e.g. division of one string by another.

### Julia matrix types

Julia has a wide variety of predefined matrix types. For example:

In [16]:
D = Diagonal([1, 2, 3.])

3×3 Diagonal{Float64,Array{Float64,1}}:
 1.0   ⋅    ⋅ 
  ⋅   2.0   ⋅ 
  ⋅    ⋅   3.0

This is an efficient way to store a diagonal matrix: only the diagonal elements are actually stored, in `D.diag`:

In [17]:
D.diag

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

The `Diagonal` type has sensible built-in behaviour; for example, we cannot set off-diagonal elements (which must be 0):

In [18]:
D[1,2] = 10

ArgumentError: ArgumentError: cannot set off-diagonal entry (1, 2) to a nonzero value (10)

but we can modify diagonal elements:

In [19]:
D[2,2] = 10

10

In [40]:
D

3×3 Diagonal{Float64}:
 1.0    ⋅    ⋅ 
  ⋅   10.0   ⋅ 
  ⋅     ⋅   3.0

Multiplication of a `Diagonal` matrix by a (dense) vector is defined:

In [20]:
v = [1, 2, 3.]
D*v

3-element Array{Float64,1}:
  1.0
 20.0
  9.0

We can check which method is actually being called here:

In [43]:
@which D*v

*(D::Diagonal, V::AbstractArray{T<:Any,1}) at linalg/diagonal.jl:117

By clicking through to the source code, we see that the definition of this method is very simple. 

We can create a dense (standard Julia matrix) version of `D`:

In [22]:
Matrix(D)  # actual dense Julia matrix

3×3 Array{Float64,2}:
 1.0   0.0  0.0
 0.0  10.0  0.0
 0.0   0.0  3.0

and can write a test to see if the two ways of multiplying agree; make sure to always do tests like this when you define your own types:

In [24]:
using Test
@test D*v == Matrix(D)*v   # use ≈ (type as \approx<TAB>) for approximate equality 

Test Passed

Since `*` is defined for `D` and vectors `v`, we can now just apply our `power_method`:

In [25]:
power_method(D, [1,1.,0.])

([1.0e-100, 1.0, 0.0], 10.0)

Another predefined Julia matrix type is `SymTriDiagonal`, which is an efficient representation for symmetric, tridiagonal matrices, storing only the diagonal and super-diagonal elements:

In [27]:
n = 100
S = SymTridiagonal(rand(n), rand(n-1))
S[1:5, 1:5]

5×5 SparseArrays.SparseMatrixCSC{Float64,Int64} with 13 stored entries:
  [1, 1]  =  0.752478
  [2, 1]  =  0.401048
  [1, 2]  =  0.401048
  [2, 2]  =  0.584996
  [3, 2]  =  0.419795
  [2, 3]  =  0.419795
  [3, 3]  =  0.34204
  [4, 3]  =  0.362963
  [3, 4]  =  0.362963
  [4, 4]  =  0.919046
  [5, 4]  =  0.912341
  [4, 5]  =  0.912341
  [5, 5]  =  0.670095

Again, since matrix-vector multiplication with `*` is already defined, we can immediately apply our function:

In [28]:
vec, val = power_method(S, rand(n))

([4.32054e-8, 1.18655e-7, 3.1737e-7, 1.18469e-6, 1.08763e-6, 3.56635e-7, 7.669e-9, 3.11799e-9, 2.28126e-9, 9.4747e-10  …  0.72416, 0.609692, 0.0629889, 0.0024887, 0.000932654, 5.93432e-5, 2.09839e-5, 5.51604e-6, 1.46514e-7, 3.87145e-8], 2.122353903853173)

Let's compare the result with the built-in Julia function `eigvals`, which, in this case, uses a highly optimized function from the LAPACK library:

In [30]:
λ = eigvals(S)
val - maximum(abs.(λ))

-5.3635314534172807e-5

Indeed we found a good approximation to the top eigenvalue; a better approximation would require to iterate longer until convergence.

We may want to restrict the allowed types that we pass in to the function, e.g. by using the type signature

    function power_method(M, v::AbstractVector)

so that we can only pass objects of type `AbstractVector` as the second argument. On the other hand, we may lose some genericity like this.

### BigFloats

Julia includes a library for arbitrary-precision calculations, `BigFloat`.

In [31]:
M = BigFloat[2 1; 1 1.]

2×2 Array{BigFloat,2}:
 2.0  1.0
 1.0  1.0

The default is 256 binary digits (bits) of precision:

In [32]:
precision(M[1,1]) 

256

which is about 77 decimal digits:

In [33]:
log10(2)*precision(M[1,1])

77.06367888997919

This can be changed with

In [34]:
setprecision(BigFloat, 128)

128

We can now find an eigenvalue to high precision using the *same* algorithm:

In [35]:
v = [big(1.0), big(1.0)]

2-element Array{BigFloat,1}:
 1.0
 1.0

In [36]:
power_method(M, v)

(BigFloat[8.50651e-01, 5.25731e-01], 2.61803398874989484820458683436563811772)

For this particular matrix, an analytical expression for the eigenvalue may be obtained in the standard way, as a root of the characteristic polynomial $(\lambda - 2)(\lambda - 1) - 1 = 0$, giving

In [37]:
λ = (3 + sqrt(big(5))) / 2

2.61803398874989484820458683436563811772

In [38]:
vec, val = power_method(M, v)

(BigFloat[8.50651e-01, 5.25731e-01], 2.61803398874989484820458683436563811772)

In [39]:
λ - val

0.0